# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1449442302 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1449442302 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to zg33@columbia.edu and will expire on October 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-11807 - Server binary: /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1449442302.log
[INFO] GraphLab Server Version: 1.7.1


In [29]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

# Features are:
* `sqft_living`
* `bedrooms`
* `bathrooms`

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Number of coefficients    : 4
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.046473     | 4146407.600631     | 258679.804477 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


In [14]:
print("max_iterations: %s" % example_model.max_iterations)
print("training_iterations %s" % example_model.training_iterations)
print("step_size %s: %s" % (u'\u03B7', example_model.step_size))
print("convergence_threshold %s: %s" % (u'\u03B5', example_model.convergence_threshold))
print("coefficients %s:\n %s" % ("w", example_model.coefficients))
example_model.summary

max_iterations: 10
training_iterations 1
step_size η: 1.0
convergence_threshold ε: 0.01
coefficients w:
 +-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | 87910.0724924  |
| sqft_living |  None | 315.403440552  |
|   bedrooms  |  None | -65080.2155528 |
|  bathrooms  |  None | 6944.02019265  |
+-------------+-------+----------------+
[4 rows x 3 columns]



<bound method LinearRegression.summary of Class                         : LinearRegression

Schema
------
Number of coefficients        : 4
Number of examples            : 17384
Number of feature columns     : 3
Number of unpacked features   : 3

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 1.0627

Settings
--------
Residual sum of squares       : 1.16325455379e+15
Training RMSE                 : 258679.8045

Highest Positive Coefficients
-----------------------------
(intercept)                   : 87910.0725
bathrooms                     : 6944.0202
sqft_living                   : 315.4034

Lowest Negative Coefficients
----------------------------
bedrooms                      : -65080.2156
>

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [15]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | 87910.0724924  |
| sqft_living |  None | 315.403440552  |
|   bedrooms  |  None | -65080.2155528 |
|  bathrooms  |  None | 6944.02019265  |
+-------------+-------+----------------+
[4 rows x 3 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this notebook we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [16]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [18]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    
    # Then compute the residuals/errors
    residuals = outcome - predictions

    # Then square and add them up
    RSS = (residuals * residuals).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [19]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [27]:
from math import log
log(2) # natural logorithm of 2
import math
log(math.e)

1.0

Next create the following 4 new features as column in both TEST and TRAIN data:

# Features are:
* `bedrooms_squared = bedrooms * bedrooms`
* `bed_bath_rooms = bedrooms * bathrooms`
* `log_sqft_living = log(sqft_living)`
* `lat_plus_long = lat + long` 

As an example here's the first one:

In [21]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [42]:
# create the remaining 3 features in both TEST and TRAIN data
# bad_bath_rooms
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['bed_bath_rooms']

dtype: float
Rows: 4229
[3.0, 12.0, 6.0, 7.5, 3.0, 10.0, 9.0, 10.0, 5.25, 8.0, 5.25, 10.0, 10.0, 5.25, 8.75, 10.0, 6.75, 6.75, 12.5, 3.0, 4.0, 3.0, 11.25, 6.0, 10.0, 5.25, 10.0, 7.5, 7.5, 7.5, 3.0, 7.0, 7.5, 2.0, 6.0, 4.0, 9.0, 20.0, 10.0, 6.0, 6.75, 0.75, 3.0, 7.0, 2.0, 15.0, 10.0, 9.0, 10.0, 6.0, 7.5, 6.75, 13.0, 13.0, 2.0, 6.75, 6.0, 2.0, 8.0, 2.0, 8.0, 8.25, 5.25, 5.25, 10.0, 6.0, 0.75, 11.0, 2.0, 4.0, 12.5, 10.0, 12.0, 9.0, 5.25, 9.0, 10.0, 8.25, 7.5, 11.0, 11.25, 2.0, 10.0, 9.75, 6.0, 5.25, 9.0, 2.0, 6.0, 12.0, 10.0, 4.5, 3.0, 5.25, 5.25, 6.75, 10.0, 6.0, 4.0, 2.0, ... ]

In [47]:
# create the remaining 3 features in both TEST and TRAIN data
# log_sqft_living
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))


In [39]:
# create the remaining 3 features in both TEST and TRAIN data
# lat_plus_long
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [48]:
test_data['log_sqft_living'].mean()

7.550274679645938

In [49]:
four_new_features = ["bedrooms_squared", "bed_bath_rooms", "log_sqft_living", "lat_plus_long"]
four_new_features
Q2_2_A1_TEST_data_means = {x: test_data[x].mean() for x in four_new_features}
Q2_2_A1_TEST_data_means

{'bed_bath_rooms': 7.503901631591394,
 'bedrooms_squared': 12.446677701584301,
 'lat_plus_long': -74.65333497217307,
 'log_sqft_living': 7.550274679645938}

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [50]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [51]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target='price', features=model_1_features, 
                                                  validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target='price', features=model_2_features, 
                                                  validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target='price', features=model_3_features, 
                                                  validation_set = None)


PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 5
PROGRESS: Number of unpacked features : 5
PROGRESS: Number of coefficients    : 6
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.069870     | 4074878.213096     | 236378.596455 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROG

In [76]:
# Examine/extract each model's coefficients:
vars()["model_1"]
Q2_2_A2_coefficients = [ (vars()["model_%d" % i]).coefficients for i in range(1,4) ]
Q2_2_A2_coefficients = { ("model %d" % i) : Q2_2_A2_coefficients[i-1] for i in range(1,4)}
Q2_2_A2_coefficients

{'model 1': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: 6
 
 Data:
 +-------------+-------+----------------+
 |     name    | index |     value      |
 +-------------+-------+----------------+
 | (intercept) |  None | -56140675.7444 |
 | sqft_living |  None | 310.263325778  |
 |   bedrooms  |  None | -59577.1160683 |
 |  bathrooms  |  None | 13811.8405419  |
 |     lat     |  None | 629865.789485  |
 |     long    |  None | -214790.285186 |
 +-------------+-------+----------------+
 [6 rows x 3 columns], 'model 2': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: 7
 
 Data:
 +----------------+-------+----------------+
 |      name      | index |     value      |
 +----------------+-------+----------------+
 |  (intercept)   |  None | -54410676.1152 |
 |  sqft_living   |  None | 304.449298056  |
 |    bedrooms    |  None | -116366.04323  |
 |   bathrooms    |  None | -77972.3305131 |
 |      lat       |  None | 625433.834953  |
 |      long      |  None | -203958.602959

In [82]:
c = Q2_2_A2_coefficients["model 1"]
c[c['name'] == 'bathrooms']
Q2_2_A2_bathrooms = {x: Q2_2_A2_coefficients[x] [Q2_2_A2_coefficients[x]['name'] == 'bathrooms']
                        for x in Q2_2_A2_coefficients}
Q2_2_A2_bathrooms

{'model 1': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+---------------+
 |    name   | index |     value     |
 +-----------+-------+---------------+
 | bathrooms |  None | 13811.8405419 |
 +-----------+-------+---------------+
 [? rows x 3 columns]
 Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
 You can use len(sf) to force materialization., 'model 2': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+----------------+
 |    name   | index |     value      |
 +-----------+-------+----------------+
 | bathrooms |  None | -77972.3305131 |
 +-----------+-------+----------------+
 [? rows x 3 columns]
 Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
 You can use len(sf) to force materialization., 'model 3': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+---------------+
 |    name   | i

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [90]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
Q2_2_A3_RSS_TRAIN = [get_residual_sum_of_squares((vars()["model_%d" % i]), train_data, train_data['price']) 
               for i in range(1,4)]
for x in Q2_2_A3_RSS_TRAIN:
    print(str(x))

9.71328233544e+14
9.61592067856e+14
9.05276314556e+14


In [91]:
Q2_2_A3_RSS_TRAIN_interpretation = """model_3 is best, and this is not surprising as
its features are the superset of all the other models"""
Q2_2_A3_RSS_TRAIN_interpretation

'model_3 is best, and this is not surprising as\nits features are the superset of all the other models'

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [92]:
# Compute the RSS on TESTING data for each of the three models and record the values:
Q2_2_A4_RSS_TEST = [get_residual_sum_of_squares((vars()["model_%d" % i]), test_data, test_data['price']) 
               for i in range(1,4)]
for x in Q2_2_A4_RSS_TEST:
    print(str(x))

2.26568089093e+14
2.24368799994e+14
2.5182931895e+14


In [95]:
Q2_1_A4_RSS_TEST_interpretation = """
Model #2 has the best validated results. It looks like the first model could have used
some additional variables, while the last model was overtrained. In particular,
adding the lat + long added extra noise (or it could have been the other two, that is not clear)
"""

print(Q2_1_A4_RSS_TEST_interpretation)


Model #2 has the best validated results. It looks like the first model could have used
some additional variables, while the last model was overtrained. In particular,
adding the lat + long added extra noise (or it could have been the other two, that is not clear)



**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [96]:
Q2_2_A1_TEST_data_means

{'bed_bath_rooms': 7.503901631591394,
 'bedrooms_squared': 12.446677701584301,
 'lat_plus_long': -74.65333497217307,
 'log_sqft_living': 7.550274679645938}

In [98]:
Q2_2_A2_bathrooms

{'model 1': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+---------------+
 |    name   | index |     value     |
 +-----------+-------+---------------+
 | bathrooms |  None | 13811.8405419 |
 +-----------+-------+---------------+
 [? rows x 3 columns]
 Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
 You can use len(sf) to force materialization., 'model 2': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+----------------+
 |    name   | index |     value      |
 +-----------+-------+----------------+
 | bathrooms |  None | -77972.3305131 |
 +-----------+-------+----------------+
 [? rows x 3 columns]
 Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
 You can use len(sf) to force materialization., 'model 3': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +-----------+-------+---------------+
 |    name   | i

In [99]:
Q2_2_A3_RSS_TRAIN

[971328233543691.8, 961592067855796.5, 905276314555551.8]

# 8 / 8 CORRECT!!!